In [ ]:
# Classes File

In [2]:
# Local Import
import numpy as np
import pandas as pd
import math
import scipy.stats as ss
import numpy.linalg as la
from itertools import product
import numpy.random as nr
import pickle

# Plot Tools
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import cycler
colors = cycler('color', 
       ['#EE6666', '#3388BB', '#9988DD', '#EECC55', 
       '#88BB44', '#FFBBBB'])

plt.rc('axes', facecolor='#E6E6E6', edgecolor='none',
      axisbelow=True, grid=True, prop_cycle=colors)
plt.rc('grid', color='w', linestyle='solid')
plt.rc('xtick', direction='out', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('patch', edgecolor='#E6E6E6')
plt.rc('lines', linewidth=2)

In [4]:
# R in Python Interface
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri as rnp
robustbase = importr('robustbase')
base = importr('base')
utils = importr('utils')

In [ ]:
#diffpriv = importr('diffpriv')
# Run this in an R console if it ever disappears
# install.packages('diffpriv')
# install.packages("devtools")
# devtools::install_github("brubinstein/diffpriv")

In [ ]:
# OBJECTS

In [6]:
def npl(x):
    ''' Convert subarrays into lists 
    '''
    return np.asarray([i.tolist() for i in x])

In [8]:
class Wrangle:
    ''' Data Wrangling Class for Test Data 
    '''
    def __init__(self, data, aggregate=False):
        ''' Numpy-nize Census Tract 
        '''
        if 'PUMA5' in data:
            data = data.drop(['PUMA5'], axis=1)
        if data.keys()[0] != 'Cell':
            data = data[data.columns[::-1]]
        self.Key = data.keys()
        if aggregate:
            Y = np.asarray(data.loc[:, self.Key[1]])
            X = np.asarray(data.loc[:, self.Key[2]])
            self.N = len(X)
        else:
            if len(self.Key) > 3:
                None
            else:
                Cell = np.unique(np.array(data[self.Key[0]])).astype(int)
                Y = [np.asarray(data.loc[data.loc[:, self.Key[0]]== i, self.Key[1]]) for i in Cell]
                X = [np.asarray(data.loc[data.loc[:, self.Key[0]]== i, self.Key[2]]) for i in Cell]
                self.N = np.asarray([len(i) for i in X])
        self.X = X
        self.Y = Y 
        self.v = len(self.Key) - 1
                
    def __call__(self):
        return self.X, self.Y, self.N, self.v

In [59]:
class Methods:
    ''' Class of regression methods and their parts
    '''
    def __init__(self, X, Y, N, v, method=[]):
        ''' Make sure you are using at most 2-dimensional arrays for X, Y
        '''
        # We assume X is organised by index with dependent variables inside each
        self.X = np.asarray(X)
        self.Y = np.asarray(Y)
        self.method = method
        if "SMDM" in self.method:
            self.form = ro.Formula("y~x")
        self.N = N
        self.v = v
        self.o = np.asarray([list(i) for i in product([0, 1], repeat = self.v)])
        if type(self.N) is int:
            self.index = [None]
        else:
            self.index = range(len(self.N))
        self.range = range(4) 
        self.nd75 = ss.norm.ppf(0.75)
        if "Winsorize" in self.method:
            self.win = True
        else:
            self.win = False
        if "SE" in self.method:
            self.SE = True
        else:
            self.SE = False
    
    def sort(self, I=None, O=None, win=False):
        ''' Sort arrays and append outlier variations
        '''
        if I is None:
            y = self.Y
            x = self.X
        else:
            y = self.Y[I]
            x = self.X[I]
        if O is not None:
            y = np.block([y, self.o[O][0]])
            x = np.block([x, self.o[O][1]])
        if win:
            x = ss.mstats.winsorize(x, limits=0.05)
            x = x.filled()
            y = ss.mstats.winsorize(y, limits=0.05)
            y = y.filled()
        return x, y
    
    def stack(self, x):
        ''' Add constant to estimate 
        '''
        return np.block([[np.ones(len(x))], [x.T]]).T
    
    def HCE(self, X, r, a):
        ''' Calculate Heteroscedasticity-consistent standard errors
        '''
        HCE = np.dot(np.dot(a, X.T.dot(np.dot(np.diag(np.square(r)), X))), a)
        return np.array(np.diag(HCE))
    
    def lstsq(self, x, y, w=None, se=False):
        ''' Calculus linear least-squares
        '''
        x = self.stack(x)
        if w is not None:
            X = x.T.dot(np.diag(w)).T
        else:
            X = x
        a = la.inv(X.T.dot(x))
        β̂ = np.dot(a, X.T.dot(y))
        r = y - np.dot(X, β̂)
        if se:
            se = self.HCE(X, r, a)
            #se = np.sqrt(r.dot(r) / np.sum(np.square(X.T[1] - np.mean(X.T[1]))) / (len(r) - self.v))
        return β̂, r, se

    # OLS Method:
    def OLS(self, I=None, O=None, se=False):
        x, y = self.sort(I=I, O=O, win=self.win)
        β̂, r, se = self.lstsq(x, y, None, se)
        return β̂, r, se
        
    # MM Method:
    def MAD(self, r):
        ''' Median Absolute Deviation 
        '''
        return np.median(np.absolute(r - np.median(r)))
    
    def Tukey(self, u, c=1.547):
        ''' Tukey's biweight function ''' 
        ρ = np.power(u, 2) / 2 - np.power(u, 4) / (2 * math.pow(c, 2)) + np.power(u, 6) / (6 * math.pow(c, 4))
        if np.any(np.abs(u) > c):
            ρ[np.abs(u) > c] = math.pow(c, 2) / 6            
        return ρ
            
    def σ(self, r, w=None, K=0.199):
        ''' The scale for the S-Estimator we wish to minimize
        '''
        if w is None:
            σ = self.MAD(r) / self.nd75 
        else:
            σ = np.sqrt(np.sum(np.multiply(np.square(r),w)) / (len(r) * K))
        return σ
    
    def u(self, r, σ):
        ''' Bisquare ratio 
        '''
        return r / σ
    
    def weight(self, u, method="S", it=False):
        ''' Estimator Reweighing
        '''
        if method=="S":
            if not it:
            # iteration = 1
                w = np.square(u / 1.547)
                if np.any(np.abs(u) > 1.547):
                    w[np.abs(u) > 1.547] = 0
                weight = np.square(1 - w)
            else:
                weight = self.Tukey(u) / np.square(u)
        if method=="MM":
            w = np.square(u / 4.685)
            if np.any(np.abs(u) > 4.685):
                w[np.abs(u) > 4.685] = 0
            weight = np.square(1 - w)
        return weight

    def MM(self, I=None, O=None, se=False, maxiter=100):
        ''' MM-Estimator regression algorithm as specified in Susanti 2009
        '''
        x, y = self.sort(I, O, win=self.win)
        β̂, r, se = self.lstsq(x, y, None, False)
        # 2. S-Estimation
        # 1st iteration
        σ = self.σ(r)
        u = self.u(r, σ)
        w = self.weight(u, "S", False)
        β̂, r, se = self.lstsq(x, y, w, False)
        # IRWLS
        for i in range(maxiter):
            σ = self.σ(r, w)
            u = self.u(r, σ)
            w = self.weight(u, "S", True)
            S, r, se = self.lstsq(x, y, w, False)
            if np.allclose(S, β̂, rtol=1e-09):
                break
            else:
                β̂ = S
        # We use the MAD of the residuals of the S-estimator
        σ = self.σ(r)
        # 3. MM-Estimate Convergence 
        for i in range(maxiter):
            u = self.u(r, σ)
            w = self.weight(u, "MM")
            M, r, se = self.lstsq(x, y, w, False)
            if np.allclose(M, β̂, rtol=1e-03):
                break
            else:
                β̂ = M
        if se:
            β̂s, rs, se = self.lstsq(x, y, w, True)
        return M, r, se, σ   
        
    # SMDM Method:
    def SMDM(self, I=None, O=None):
        ''' Run SMDM from R's 'lmrob' package via rpy2
        '''
        x, y = self.sort(I, O, win=self.win)
        try:
            nr, nc = x.shape
        except:
            nr = x.shape[0]
            nc = 1
        # Set up formula environment
        rnp.activate()
        form.environment["y"] = ro.r['matrix'](y, nrow=nr, ncol=1)
        form.environment["x"] = ro.r['matrix'](x, nrow=nr, ncol=nc)
        # Run Robust Regression
        lmr = robustbase.lmrob(self.form, method = "SMDM", setting="KS2014")
        # must copy variables because of memory constraint
        betas = np.array(lmr.rx2("coefficients"), copy=True)
        scale = np.array(lmr.rx2("scale"), copy=True)
        residuals = np.array(lmr.rx2("residuals"), copy=True)
        stderr = np.array(np.diag(lmr.rx2("cov")), copy=True)
        return betas, residuals, stderr, scale
    
    def __call__(self, withoutliers=True):
        ''' Specify regression method and run all regression in one/two line(s)
        '''
        if "OLS" in self.method:
            if self.index is None:
                regression = self.OLS(None, None, self.SE)
                if withoutliers:
                    withoutliers = np.asarray([self.OLS(None, O, self.SE) for O in self.range])
            else:
                regression = np.asarray([self.OLS(I, None, self.SE) for I in self.index])
                if withoutliers:
                    withoutliers = np.asarray([[self.OLS(I, O, self.SE) for O in self.range] for I in self.index])    
        elif "MM" in self.method:
            if self.index is None:
                regression = self.MM(None, None, self.SE, 100)
                if withoutliers:
                    withoutliers = np.asarray([self.MM(None, O, self.SE) for O in self.range])
            else:
                regression = np.asarray([self.MM(I, None, self.SE) for I in self.index])
                if withoutliers:
                    withoutliers = np.asarray([[self.MM(I, O, self.SE) for O in self.range] for I in self.index])
        elif "SMDM" in self.method:
            if self.index is None:
                regression = self.SMDM(None, None)
                if withoutliers:
                    withoutliers = np.asarray([self.SMDM(None, O) for O in self.range])
            else:
                regression = np.asarray([self.SMDM(I, None) for I in self.index])
                if withoutliers:
                    withoutliers = np.asarray([[self.MM(I, O) for O in self.range] for I in self.index])
        return regression, withoutliers

In [ ]:
class LocSnoises:
    ''' Release Algorithm by Chetty & Friedman
    '''
    def __init__(self, regression, withoutliers, N, method=["Laplace"]):
        self.RA = regression
        self.RB = withoutliers
        self.range = range(4)
        self.N = N
        if type(self.N) is int:
            self.index = [None]
        else:
            self.index = range(len(self.N))
        self.method = method
            
    def ω(self):
        ''' Draws from Laplace or Normal Distribution 
        '''
        if "Laplace" in self.method:
            return nr.laplace(0, 1/math.sqrt(2))
        elif "Gaussian" in self.method:
            return nr.normal(0, math.sqrt(1/math.sqrt(2)))
    
    def LS(self):
        ''' Calculate Local Sensitivity 
        '''
        MOSE = max(np.multiply(self.N, 
                [max([abs(self.RB[i][j][0][1] - self.RA[i][0][1]) for j in self.range]) for i in self.index]))
        return np.transpose([[self.RA[i][j] for j in [0, 2]] for i in self.index]), MOSE
    
    def noise(self, θseθ, χ):
        ''' Add Noise to Statistics 
        '''
        NN = np.asarray(self.N).astype(float)
        S = NN * self.ϵ
        noise = lambda x: x * math.sqrt(2)
        
        nθ = [i[1] for i in θseθ[0]] + χ * noise(self.ω()) / S
        sen = np.square(θseθ[1]) + 2 * np.square(χ / S)
        senθ = np.sqrt(sen.astype(float))
        nsenθ = senθ + χ * noise(self.ω()) / S
        nsenθ = np.asarray(["Sample Size Too Small" if i==np.inf else i for i in nsenθ])
        nN = self.N * (1 + noise(self.ω()) / S)
        return nθ, nsenθ, nN

    def MSE(self, θ):
        MSE = []
        for i in range(1,11):
            noise_θ = []
            diff = []

            # But also, this needs to be done 500 times
            for j in range(0,500):
                # Draws random samples from Laplace or Normal:
                ω = np.random.laplace(0, 1 / np.sqrt(2))

                # Noise infused Statistics
                noise_θ = [a + np.sqrt(2) * (χ / (i * b)) * ω for (a, b) in zip(θ, N)]
                diff.append(np.square(np.subtract(noise_θ, θ)))

            # Compute MSE
            return MSE.append(np.mean(diff))

    def __call__(self):
        ''' Release Noise Infused Statistics 
        '''
        m1, m2 = self.LS()
        nθ, nsenθ, nN = self.noise(m1, m2)
        return nθ, nsenθ, nN

In [ ]:
class DWORK:
    def __init__(self, Tr, r, o, ϵ, psize, method=["Laplace"]):
        ''' Dwork & Lei Release Algorithm
            Developed for releasing true β w/ noise
            Can be used for releasing SE's as well
        '''
        # length of each cell
        self.n = psize
        # base for each cell (for S algorithm)
        self.base = np.asarray([1 + 1 / np.log(i) for i in self.n])
        
        # store β̂'s and β̂ᵢ's:
        self.Tb = Tr[0][0][0]
        self.b = npl(r.T[0]).T
        self.db = npl(o.T[0].T)
        
        # number of partitions & range
        p = len(self.b.T)
        self.range = range(p)
        # total sample size
        self.N = len(Tr[0][0][1])
        self.ϵ = ϵ
        # define delta parameter
        ndivp = self.N / p
        self.δ = 0.5 * math.pow(ndivp, (- ϵ) * math.log(ndivp))
        
        self.method = method
    
    def ω(self, h=1):
        ''' Laplace of Gaussian Mechanism
        '''
        if "Laplace" in self.method:
            return nr.laplace(0, h/self.ϵ)
        elif "Gaussian" in self.method:
            return math.sqrt(h * math.sqrt(2 * math.log(1.25 / self.δ)) / self.ϵ)
        else:
            raise ValueError('No Mechanism Specified')
    
    # 2. Run S algorithm
    def IQR(self, b, db=None):
        ''' Function: Interquartile Range of partitioned β̂'s
        '''
        if db is not None:
            # Array like O array but with iqr calc for switching old β with new β
            nb = np.asarray([[np.block([[np.delete(b.T, i, 0)], [db[i][j]]]) for j in range(4)] for i in self.range])
            IQR = np.asarray([[[np.percentile(k, 75) - np.percentile(k, 25) for k in i.T] for i in j]for j in nb])
        else:
            # Original IQR for β
            IQR = np.asarray([np.percentile(i, 75) - np.percentile(i, 25) for i in b])
        return IQR
    
    def H(self, o=False):
        ''' Part 1: H
            - Compute H for each β̂ and compute H' for alt β̂'s'
        '''
        if o:
            IQR = self.IQR(self.b, self.db)
            H = np.asarray([[np.log(IQR[i][j]) / np.log(self.base[i]) for j in range(4)]                          
                           for i in self.range])
        else:
            IQR = self.IQR(self.b)
            H = np.asarray([np.log(IQR) / np.log(self.base[i]) for i in self.range])
        return H
    
    def S(self, BOOL=False):
        ''' Part 2: S Algorithm
            - Compute Noise-infused SCALE for β̂'s
        '''
        H = self.H()
        dH = self.H(True)
        # 2.3 compute bins for each H
        bins = np.asarray([[np.abs(H[i] - dH[i][j] + self.ω()) for j in range(4)] for i in self.range])
        # 2.4 return TRUE for violation of 2.3 <= 1
        booL = np.asarray([np.all(i > 1) for i in bins])
        # [optional] for seeing which cells fail
        if BOOL:
            return booL
        else:
            s = np.asarray([self.IQR(self.b) * self.base[i] ** self.ω() for i in self.range])
            s[booL] = np.inf
            return s

    # 3. compute h for each s
    def h(self, s):
        h = np.asarray([i / math.pow(self.N, 0.25) for i in s])
        h[h==0] = 1 / math.sqrt(self.N)
        h[h==np.inf] = 0
        return h
    
    def z(self, h):
        return np.asarray([
            self.ω(i) for i in h])
    
    def RH(self, s, BOOL=False):
        ''' Part 3: Release
            - Compute True β + Noise
        '''
        h = self.h(s)
        # 3.1 np.abs(alt β̂'s - β̂) <= h array
        bins = np.asarray([[np.abs(self.b.T[i] - self.db[i][j]) for j in range(4)] for i in self.range])
        booL = np.asarray([[bins[i][j] > h[i] for j in range(4)] for i in self.range])
        # return True for violation
        anybooL = np.asarray([not np.any([not np.any(booL[i][j]) for j in range(4)]) for i in self.range])
        # 3.2 for TRUE compute β + noise
        RHb = self.Tb + self.z(h)
        RHb[anybooL] = None
        # [optional] for seeing which cells fail
        if BOOL:
            return anybooL
        else:
            try:
                # 3.3 find min(β + noise)
                np.nanmin(RHb, 0)
                return np.nanmin(RHb, 0)
            except:
                return None
            
    def __call__(self, BOOL=False):
        ''' Set up so one can return Boolean Arrays for both S and RH algorithm 
        '''
        if BOOL:
            s = self.S()
            RH = self.RH(s, BOOL)
            S = self.S(BOOL)
            return S, RH
        else:
            s = self.S()
            RH = self.RH(s, BOOL)
            if RH is None:
                return "Too Sensitive"
            else:
                return s, RH

In [69]:
class MSE:
    ''' We compute MSE's for the DWORK algorithm outputs for different epsilons
    '''
    def __init__(self, Tr, r, o, psize, method=["Laplace"]):    
        self.T = Tr[0][0][0]
        self.Tr = Tr
        self.r = r
        self.o = o
        self.psize = psize
        self.method = method
        
    def __call__(self):
        self.MSE = []
        for ϵ in range(1,11):
            diff = []
            # We repeat DWORK 20 times
            for j in range(0,20):
                s, noise = DWORK(self.Tr, self.r, self.o, ϵ, self.psize, self.method)
                try:
                    diff.append(np.square(np.subtract(noise, self.T)))
                except:
                    pass
            # Compute MSE
        return self.MSE.append(np.mean(diff))
    

In [ ]:
# Tests and Data Storage

In [12]:
# Test 1 Using Alaska V v. H
%cd /home/nbuser/library/DP/Data/
with open("Alaska_VH.pickle", "rb") as handle:
    VH = pickle.load(handle)
with open("Alaska_VH.pickle", "rb") as handle:
    YA = pickle.load(handle)

/home/nbuser/library/DP/Data


In [63]:
VH101 = Wrangle(VH.get_group(101))
vh101X, vh101Y, vh101N, vh101v = VH101()
VHOLS = Methods(vh101X, vh101Y, vh101N, vh101v, ["OLS", "Winsorize", "SE"])

In [65]:
Or = VHOLS(False)

In [ ]:
x, y, n, v = Wrangle(data)()
hey = Methods(x, y, n, v, "OLS")
Q0, Q1 = hey.sort(0)
Q2, Q3 = hey.winsorize(Q0, Q1)

In [ ]:
XX, YY, NN, vv = Wrangle(data, aggregate=True)()
X, Y, N, v = Wrangle(data)()

In [ ]:
OLSA = Methods(XX, YY, NN, vv, "OLS")
OLSP = Methods(X, Y, N, v, "OLS")

MMA = Methods(XX, YY, NN, vv, "MM")
MMP = Methods(X, Y, N, v, "MM")

SMDMA = Methods(XX, YY, NN, vv, "SMDM")
SMDMP = Methods(X, Y, N, v, "SMDM")

In [ ]:
# 0. calc true statistic (aggregate level)
# 1. store partition stats (cell/tract level)
OTr = np.asarray(OLSA(withoutliers=False))
Or, Oo = OLSP()
Opsize = OLSP.N

MTr = np.asarray(MMA(withoutliers=False))
Mr, Mo = MMP()
Mpsize = mP.N

STr = np.asarray(SMDMA(withoutliers=False))
Sr, So = SMDMP()
Spsize = SMDMP.N

In [ ]:
#pickle.dump([OTr, Or, Oo, Opsize, MTr, Mr, Mo, Mpsize, STr, Sr, So, Spsize], open("trial.p", "wb"))
# a1, b1, c1, d1, a2, b2, c2, d2, c3, b3, c3, d3  = pickle.load(open("trial.p","rb"))

In [ ]:
OLSDWORK = DWORK(OTr, Or, Oo, 4, Opsize)
MMDWORK = DWORK(MTr, Mr, Mo, 4, Mpsize)
SMDMDWORK = DWORK(STr, Sr, So, 4, Spsize)

In [ ]:
OD = OLSDWORK()
OB = OLSDWORK(True)
MD = MMDWORK()
MB = MMDWORK(True)
SD = SMDMDWORK()
SB = SMDMDWORK(True)

In [ ]:
# Timeit MM v SMDM

In [ ]:
t = OTr[0][0][0]
print(abs(t - OD))
print(abs(t - MD))
print(abs(t - SD))

In [ ]:
print(np.sum(OB[0]), np.sum(OB[1]))
print(np.sum(MB[0]), np.sum(MB[1]))
print(np.sum(SB[0]), np.sum(SB[1]))